In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch.utils.data as data
import torch
import h5py

class DatasetFromHdf5(data.Dataset):
    def __init__(self, file_path):
        super(DatasetFromHdf5, self).__init__()
        hf = h5py.File(file_path)
        self.data = hf.get('data')
        self.target = hf.get('label')

    def __getitem__(self, index):
        return torch.from_numpy(self.data[index,:,:,:]).float(), torch.from_numpy(self.target[index,:,:,:]).float()

    def __len__(self):
        return self.data.shape[0]

In [3]:
import torch
import torch.nn as nn
import math

class MeanShift(nn.Conv2d):
    def __init__(self, rgb_mean, sign):
        super(MeanShift, self).__init__(3, 3, kernel_size=1)
        self.weight.data = torch.eye(3).view(3, 3, 1, 1)
        self.bias.data = float(sign) * torch.Tensor(rgb_mean)

        # Freeze the MeanShift layer
        for params in self.parameters():
            params.requires_grad = False

class _Residual_Block(nn.Module): 
    def __init__(self):
        super(_Residual_Block, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False)

    def forward(self, x): 
        identity_data = x
        output = self.relu(self.conv1(x))
        output = self.conv2(output)
        output *= 0.1
        output = torch.add(output,identity_data)
        return output 

class EDSR(nn.Module):
    def __init__(self):
        super(EDSR, self).__init__()

        rgb_mean = (0.4488, 0.4371, 0.4040)
        self.sub_mean = MeanShift(rgb_mean, -1)

        self.conv_input = nn.Conv2d(in_channels=3, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False)

        self.residual = self.make_layer(_Residual_Block, 32)

        self.conv_mid = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False)

        self.upscale4x = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=256*4, kernel_size=3, stride=1, padding=1, bias=False),
            nn.PixelShuffle(2),
            nn.Conv2d(in_channels=256, out_channels=256*4, kernel_size=3, stride=1, padding=1, bias=False),
            nn.PixelShuffle(2),
        )

        self.conv_output = nn.Conv2d(in_channels=256, out_channels=3, kernel_size=3, stride=1, padding=1, bias=False)

        self.add_mean = MeanShift(rgb_mean, 1)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                if m.bias is not None:
                    m.bias.data.zero_()

    def make_layer(self, block, num_of_layer):
        layers = []
        for _ in range(num_of_layer):
            layers.append(block())
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.sub_mean(x)
        out = self.conv_input(out)
        residual = out
        out = self.conv_mid(self.residual(out))
        out = torch.add(out,residual)
        out = self.upscale4x(out)
        out = self.conv_output(out)
        out = self.add_mean(out)
        return out

In [4]:
import torch
import numpy as np
from scipy.ndimage import gaussian_filter


def calc_patch_size(func):
    def wrapper(args):
        if scale == 2:
            patch_size = 10
        elif scale == 3:
            patch_size = 7
        elif scale == 4:
            patch_size = 6
        else:
            raise Exception('Scale Error', scale)
        return func(args)
    return wrapper


def convert_rgb_to_y(img, dim_order='hwc'):
    if dim_order == 'hwc':
        return 16. + (64.738 * img[..., 0] + 129.057 * img[..., 1] + 25.064 * img[..., 2]) / 256.
    else:
        return 16. + (64.738 * img[0] + 129.057 * img[1] + 25.064 * img[2]) / 256.


def convert_rgb_to_ycbcr(img, dim_order='hwc'):
    if dim_order == 'hwc':
        y = 16. + (64.738 * img[..., 0] + 129.057 * img[..., 1] + 25.064 * img[..., 2]) / 256.
        cb = 128. + (-37.945 * img[..., 0] - 74.494 * img[..., 1] + 112.439 * img[..., 2]) / 256.
        cr = 128. + (112.439 * img[..., 0] - 94.154 * img[..., 1] - 18.285 * img[..., 2]) / 256.
    else:
        y = 16. + (64.738 * img[0] + 129.057 * img[1] + 25.064 * img[2]) / 256.
        cb = 128. + (-37.945 * img[0] - 74.494 * img[1] + 112.439 * img[2]) / 256.
        cr = 128. + (112.439 * img[0] - 94.154 * img[1] - 18.285 * img[2]) / 256.
    return np.array([y, cb, cr]).transpose([1, 2, 0])


def convert_ycbcr_to_rgb(img, dim_order='hwc'):
    if dim_order == 'hwc':
        r = 298.082 * img[..., 0] / 256. + 408.583 * img[..., 2] / 256. - 222.921
        g = 298.082 * img[..., 0] / 256. - 100.291 * img[..., 1] / 256. - 208.120 * img[..., 2] / 256. + 135.576
        b = 298.082 * img[..., 0] / 256. + 516.412 * img[..., 1] / 256. - 276.836
    else:
        r = 298.082 * img[0] / 256. + 408.583 * img[2] / 256. - 222.921
        g = 298.082 * img[0] / 256. - 100.291 * img[1] / 256. - 208.120 * img[2] / 256. + 135.576
        b = 298.082 * img[0] / 256. + 516.412 * img[1] / 256. - 276.836
    return np.array([r, g, b]).transpose([1, 2, 0])


def preprocess(img, device):
    img = np.array(img).astype(np.float32)
    ycbcr = convert_rgb_to_ycbcr(img)
    x = ycbcr[..., 0]
    x /= 255.
    x = torch.from_numpy(x).to(device)
    x = x.unsqueeze(0).unsqueeze(0)
    return x, ycbcr


def calc_psnr(img1, img2):
    return 10. * torch.log10(1. / torch.mean((img1 - img2) ** 2))


def calc_ssim(img1, img2, sd=1.5, C1=0.01**2, C2=0.03**2):
    img1 = img1.cpu()
    img2 = img2.cpu()
    mu1 = gaussian_filter(img1, sd)
    mu2 = gaussian_filter(img2, sd)
    mu1_sq = mu1 * mu1
    mu2_sq = mu2 * mu2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = gaussian_filter(img1 * img1, sd) - mu1_sq
    sigma2_sq = gaussian_filter(img2 * img2, sd) - mu2_sq
    sigma12 = gaussian_filter(img1 * img2, sd) - mu1_mu2
    
    ssim_num = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2))
    ssim_den = ((mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2))
    ssim_map = ssim_num / ssim_den
    mssim = np.mean(ssim_map)
    
    return mssim


class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [5]:
import os
import torch
import math, random
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader


def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10"""
    lr = lr * (0.1 ** (epoch // step))
    return lr

def train(training_data_loader, optimizer, model, criterion, epoch):

    lr = adjust_learning_rate(optimizer, epoch-1)
    
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr  

    print("Epoch={}, lr={}".format(epoch, optimizer.param_groups[0]["lr"]))
    model.train()

    for iteration, batch in enumerate(training_data_loader, 1):

        input, target = Variable(batch[0]), Variable(batch[1], requires_grad=False)

        if cuda:
            input = input.cuda()
            target = target.cuda()

        loss = criterion(model(input), target)

        optimizer.zero_grad()
        
        loss.backward()

        optimizer.step()

        if iteration%100 == 0:
            print("===> Epoch[{}]({}/{}): Loss: {:.10f}".format(epoch, iteration, len(training_data_loader), loss.data[0]))

def save_checkpoint(model, epoch):
    model_folder = "checkpoint/"
    model_out_path = model_folder + "model_epoch_{}.pth".format(epoch)
    state = {"epoch": epoch ,"model": model}
    if not os.path.exists(model_folder):
        os.makedirs(model_folder)

    torch.save(state, model_out_path)

    print("Checkpoint saved to {}".format(model_out_path))


def train_main(batchSize=16, nEpochs=500, lr=1e-4, step=200, resume='', seed=123, start_epoch=1, threads=1, momentum=0.9, weight_decay=1e-4):
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed(seed)

    cudnn.benchmark = True
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    print("===> Loading datasets")
    train_set = DatasetFromHdf5("path_to_dataset.h5")
    training_data_loader = DataLoader(dataset=train_set, num_workers=threads, batch_size=batchSize, shuffle=True)

    print("===> Building model")
    model = Net()
    criterion = nn.L1Loss(size_average=False)

    print("===> Setting GPU")
    if cuda:
        model = model.cuda()
        criterion = criterion.cuda()

    # optionally resume from a checkpoint
    if resume:
        if os.path.isfile(resume):
            print("=> loading checkpoint '{}'".format(resume))
            checkpoint = torch.load(resume)
            start_epoch = checkpoint["epoch"] + 1
            model.load_state_dict(checkpoint["model"].state_dict())
        else:
            print("=> no checkpoint found at '{}'".format(resume))

    print("===> Setting Optimizer")
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, weight_decay=weight_decay, betas = (0.9, 0.999), eps=1e-08)

    print("===> Training")
    for epoch in range(start_epoch, nEpochs + 1): 
        train(training_data_loader, optimizer, model, criterion, epoch)
        save_checkpoint(model, epoch)


In [6]:
import torch
import torch.backends.cudnn as cudnn
import numpy as np
import PIL.Image as pil_image



from torch.autograd import Variable

def test(weights_file, image_file, scale, save=False, debug=False, B=1, U=9, num_features=128):
    cudnn.benchmark = True
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    model = EDSR()
    model.load_state_dict(torch.load(weights_file, map_location=device))

    model.eval()
    model.to(device)

    image = pil_image.open(image_file).convert('RGB')
    image_file = os.path.basename(image_file)

    image_width = (image.width // scale) * scale
    image_height = (image.height // scale) * scale

    hr = image.resize((image_width, image_height), resample=pil_image.BICUBIC)
    lr = hr.resize((hr.width // scale, hr.height // scale), resample=pil_image.BICUBIC)
    bicubic = lr.resize((lr.width * scale, lr.height * scale), resample=pil_image.BICUBIC)

    hr = np.array(hr).astype(float)
    bicubic = np.array(bicubic).astype(float)
    im_l = np.array(lr).astype(float)

    hr, _ = preprocess(hr, device)
    _, ycbcr = preprocess(bicubic, device)

    im_input = im_l.astype(np.float32).transpose(2,0,1)
    im_input = im_input.reshape(1,im_input.shape[0],im_input.shape[1],im_input.shape[2])
    im_input = Variable(torch.from_numpy(im_input/255.).float()).to(device)

    with torch.no_grad():
        preds = model(im_input)

    preds = preds.cpu().data[0].numpy().astype(np.float32)
    preds = preds*255.
    preds = np.clip(preds, 0., 255.)
    preds = preds.transpose(1,2,0).astype(np.float32)

    preds, _ = preprocess(preds, device)

    psnr = calc_psnr(hr, preds)
    ssim = calc_ssim(hr, preds)

    if debug:
        print(f'PSNR/SSIM: {psnr:.2f}/{ssim:.4f}')

    preds = preds.mul(255.0).cpu().numpy().squeeze(0).squeeze(0)

    output = np.array([preds, ycbcr[..., 1], ycbcr[..., 2]]).transpose([1, 2, 0])
    output = np.clip(convert_ycbcr_to_rgb(output), 0.0, 255.0).astype(np.uint8)
    output = pil_image.fromarray(output)
    if save:
        save_path = f'/content/drive/Shareddrives/BTP Meets/results/Set5/{scale}x/{image_file}'
        output.save(save_path.replace('.', '_edsr.'))
    return float(psnr), float(ssim)


In [7]:
import os

def do_test(psnr, ssim, BASE_DIR, save=False, debug=False):
    scales = [4]

    for file in os.listdir(BASE_DIR):
        if file.endswith(".png"):
            image_file_path = os.path.join(BASE_DIR, file)
            if debug:
                print(file)
            for scale in scales:
                if debug:
                    print(f"Scale: {scale}")
                result = test(f'/content/drive/Shareddrives/BTP Meets/models/weights-edsr.pth', image_file_path, scale, save, debug)
                if scale not in psnr:
                    psnr[scale] = []
                if scale not in ssim:
                    ssim[scale] = []
                psnr[scale].append(result[0])
                ssim[scale].append(result[1])
            if debug:
                print()


In [8]:
psnr = {}
ssim = {}
do_test(psnr, ssim, '/content/drive/Shareddrives/BTP Meets/datasets/test/Set5/', True, True)

head.png
Scale: 4
PSNR/SSIM: 32.79/0.7941

butterfly.png
Scale: 4
PSNR/SSIM: 27.93/0.9086

bird.png
Scale: 4
PSNR/SSIM: 34.53/0.9374

baby.png
Scale: 4
PSNR/SSIM: 33.32/0.8901

woman.png
Scale: 4
PSNR/SSIM: 30.80/0.9179



In [ ]:
import statistics

scales = [4]
for scale in scales:
    print(f'Avg PSNR/SSIM {scale}x: {statistics.mean(psnr[scale]):.2f}/{statistics.mean(ssim[scale]):.4f}')

Avg PSNR/SSIM 4x: 31.87/0.8896


In [ ]:
scales = [4]

def calc_result(dataset):
    print()
    print(dataset)
    psnr = {}
    ssim = {}
    do_test(psnr, ssim, f'/content/drive/Shareddrives/BTP Meets/datasets/test/{dataset}/')
    for scale in scales:
        print(f'Avg PSNR/SSIM {scale}x: {statistics.mean(psnr[scale]):.2f}/{statistics.mean(ssim[scale]):.4f}')

calc_result('Set14')
calc_result('BSDS100')
calc_result('Manga109')
calc_result('Urban100')


Set14
Avg PSNR/SSIM 4x: 28.10/0.7779

BSDS100
Avg PSNR/SSIM 4x: 28.30/0.7606

Manga109
Avg PSNR/SSIM 4x: 29.99/0.9057

Urban100
Avg PSNR/SSIM 4x: 25.49/0.7691
